In [1]:
from sklearn.datasets import make_classification

In [2]:
numeric_dataset = make_classification(
    n_samples=10_000, 
    n_features=25, 
    n_informative=10,
    n_classes=2)

x, y = numeric_dataset

In [6]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=2)

In [7]:
num_categories = 100
for i in range(5):
    x[:, i] = pd.cut(x[:, i], num_categories, labels=False)

In [8]:
x[0]

array([50.  , 65.  , 61.  , 47.  , 58.  ,  0.84,  2.51, -1.11, -0.18,
       -0.91, -1.01,  1.53,  0.42,  0.89,  0.61, -0.34,  0.17, -1.08,
        3.35, -2.62, -0.39, -0.38, -4.08, -0.54,  2.06])

In [9]:
x_numeric = x[:, 5:]
x_cat = x[:, :5]

In [10]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

x_standardized = ss.fit_transform(x_numeric)

In [11]:
import tensorflow as tf

p = .1

numeric_inputs = tf.keras.layers.Input((20,), name='numeric_inputs')
cat_inputs = tf.keras.layers.Input((5,), name='cat_inputs')

In [12]:
def emb_sz_rule(n_cat): 
    return min(600, round(1.6 * n_cat**0.56))

embedding_layer = tf.keras.layers.Embedding(
    num_categories, 
    emb_sz_rule(num_categories), 
    input_length=5)
cats = embedding_layer(cat_inputs)
cats = tf.keras.layers.Flatten()(cats)

In [13]:
x = tf.keras.layers.Concatenate()([cats, numeric_inputs])

x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(10, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
out = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)

In [14]:
model = tf.keras.models.Model(
    inputs=[numeric_inputs, cat_inputs], outputs=out)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cat_inputs (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 5, 21)        2100        cat_inputs[0][0]                 
__________________________________________________________________________________________________
flatten (Flatten)               (None, 105)          0           embedding[0][0]                  
__________________________________________________________________________________________________
numeric_inputs (InputLayer)     [(None, 20)]         0                                            
_______________________________________________________________________________________

In [16]:
import numpy as np

def bootstrap_sample_generator(batch_size):
    while True:
        batch_idx = np.random.choice(
            x_standardized.shape[0], batch_size)
        yield ({'numeric_inputs': x_standardized[batch_idx],
                'cat_inputs': x_cat[batch_idx]}, 
               {'output': y[batch_idx]})

In [17]:
batch_size = 32

model.fit_generator(
    bootstrap_sample_generator(batch_size),
    steps_per_epoch=10_000 / batch_size,
    epochs=5,
    max_queue_size=10,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
313/312 [==============================] - 1s 2ms/step - loss: 0.5067 - accuracy: 0.7413
Epoch 2/5
313/312 [==============================] - 0s 1ms/step - loss: 0.3344 - accuracy: 0.8575
Epoch 3/5
313/312 [==============================] - 0s 2ms/step - loss: 0.2787 - accuracy: 0.8876
Epoch 4/5
313/312 [==============================] - 0s 1ms/step - loss: 0.2446 - accuracy: 0.9028
Epoch 5/5
313/312 [==============================] - 0s 1ms/step - loss: 0.2445 - accuracy: 0.9035
